# Tarea 1

Implementar la clase `OisCashflow`.

### Integrantes

- Daniel García-vinuesa Hödar
- Carlos Quintana Martinic

## Configuración Inicial

In [1]:
from finrisk import QC_Financial_3 as Qcf
from dataclasses import dataclass
from enum import Enum
import pandas as pd
from IPython.display import Image
from IPython.core.display import HTML
from datetime import date

# Modificado por AD
import sys
sys.path.insert(1, '../modules')
import auxiliary as aux

from functools import partial

In [2]:
class BusCal(Enum):
    NY = 1
    SCL = 2

In [3]:
def get_cal(code: BusCal) -> Qcf.BusinessCalendar:
    
    if code == BusCal.NY:
        cal = Qcf.BusinessCalendar(Qcf.QCDate(1, 1, 2020), 20)
        for agno in range(2020, 2071):
            f = Qcf.QCDate(12, 10, agno)
            if f.week_day() == Qcf.WeekDay.SAT:
                cal.add_holiday(Qcf.QCDate(14, 10, agno))
            elif f.week_day() == Qcf.WeekDay.SUN:
                cal.add_holiday(Qcf.QCDate(13, 10, agno))
            elif f.week_day() == Qcf.WeekDay.MON:
                cal.add_holiday(Qcf.QCDate(12, 10, agno))
            elif f.week_day() == Qcf.WeekDay.TUE:
                cal.add_holiday(Qcf.QCDate(11, 10, agno))
            elif f.week_day() == Qcf.WeekDay.WED:
                cal.add_holiday(Qcf.QCDate(10, 10, agno))
            elif f.week_day() == Qcf.WeekDay.THU:
                cal.add_holiday(Qcf.QCDate(9, 10, agno))
            else:
                cal.add_holiday(Qcf.QCDate(8, 10, agno))
        cal.add_holiday(Qcf.QCDate(15, 2, 2021))
        
    return cal

## `Qcf.time_series`

In [4]:
ts = Qcf.time_series()

fecha1 = Qcf.QCDate(13, 1, 1969)
ts[fecha1] = 19690113

fecha2 = Qcf.QCDate(14, 1, 1969)
ts[fecha2] = 19690114

print(f'ts[fecha1]: {ts[fecha1]}\n')

dl = Qcf.time_series_dates(ts)
for d in dl:
    print(d)
print()
    
vl = Qcf.time_series_values(ts)
for v in vl:
    print(v)

ts[fecha1]: 19690113.0

13-1-1969
14-1-1969

19690113.0
19690114.0


## Clase `OisCashflow`

La clase `OisCashflow` es una [`dataclass`](https://realpython.com/python-data-classes/). La tarea consiste en:

- Implementar los 3 métodos de la clase que están con `pass`.
- Implementar la función `present_value`.
- Implementar la función `set_expected_rate`.

### Importar la data

In [5]:
data = pd.read_excel("../data/SOFR-10012019-10292020.xls", sheet_name="nice format")
data["RATE\n(PERCENT)"]=data["RATE\n(PERCENT)"]/100
#data=pd.DataFrame(data)


### Variables

In [6]:
def _fechas(day,month,year):
    date=Qcf.QCDate(day, month, year)
    return date

In [7]:
#función para dejar las fechas en el formato requerido
def formato_año(data):
   
    data.rename(columns={"RATE\n(PERCENT)": "fixing"}, inplace=True)
    lista=[]
    i=0
    while i < (len(data["DATE"])):
        year=int(data["DATE"][i][0:4])
        month=int(data["DATE"][i][5:7])
        day=int(data["DATE"][i][8:10])
        fecha=_fechas(day, month, year)
        lista.append(fecha)
        i=i+1
    data["fechas"]=(lista)
    data=data.sort_values(by="DATE",ascending=True)
    return data
 

In [8]:
#fecha formato
data=formato_año(data)
data

,DATE,BENCHMARK NAME,fixing,fechas
269,2019-10-01,SOFR,0.0188,1-10-2019
268,2019-10-02,SOFR,0.0185,2-10-2019
267,2019-10-03,SOFR,0.0184,3-10-2019
266,2019-10-04,SOFR,0.0182,4-10-2019
265,2019-10-07,SOFR,0.0183,7-10-2019
...,...,...,...,...
4,2020-10-22,SOFR,0.0007,22-10-2020
3,2020-10-23,SOFR,0.0008,23-10-2020
2,2020-10-26,SOFR,0.0009,26-10-2020
1,2020-10-27,SOFR,0.0009,27-10-2020


In [9]:
#plazos
plazo=[1]
for j in range(0,len(data["fechas"])-1):
    plazos=data["fechas"][1+j].day_diff(data["fechas"][j])
    plazo.append(plazos)
    #plazo.append(1)
data["Plazos"]=plazo
data=data.sort_values(by="DATE",ascending=True)
data

,DATE,BENCHMARK NAME,fixing,fechas,Plazos
269,2019-10-01,SOFR,0.0188,1-10-2019,1
268,2019-10-02,SOFR,0.0185,2-10-2019,1
267,2019-10-03,SOFR,0.0184,3-10-2019,1
266,2019-10-04,SOFR,0.0182,4-10-2019,3
265,2019-10-07,SOFR,0.0183,7-10-2019,1
...,...,...,...,...,...
4,2020-10-22,SOFR,0.0007,22-10-2020,1
3,2020-10-23,SOFR,0.0008,23-10-2020,3
2,2020-10-26,SOFR,0.0009,26-10-2020,1
1,2020-10-27,SOFR,0.0009,27-10-2020,1


In [10]:
def get_wf(start_date, fecha_final, valor_tasa) -> float:
    
    qc_tasa = Qcf.QCInterestRate(valor_tasa, Qcf.QCAct360(), Qcf.QCLinearWf())
    
    return qc_tasa.wf(start_date, fecha_final)

In [11]:
#Para realizar el producto de una lista
def producto(lista_numero):
    resultado=1
    for i in lista_numero:
        resultado*=i
    return resultado

In [12]:
class OIS:
    
    # AD: esto ne es necesario. El decorador @dataclass incluye toda esta mecánica.
    # Además hay un error de sintaxis grave. El método __init__ lleva dos guiones bajos en cada lado.
    # Llama la atención que no hayan detectado este error.
    
    def _init_ (self, start_date,fixings, end_date, settlement_date, notional, currency, amortization,amort_is_cashflow,interest_rate, on_index, spread, gearing):
        
        self.start_date = start_date  
        self.fixings=fixings
        self.notional=notional
        self.currency = currency
        self.amortization = amortization
        self.amort_is_cashflow = amort_is_cashflow
        self.interest_rate = interest_rate
        self.on_index = on_index
        self.spread = spread
        self.gearing = gearing
    def get_accrued_rate(self,start_date, fixings) -> float:
        
        
        factor_capitalizacion=[]
        self.final_data=self.start_date[0]
        i=0
        while i< (len(self.start_date)-1):
            fc=get_wf(self.start_date[len(self.start_date)-1-i],self.start_date[(len(self.start_date)-2)-i],self.fixings[i])
            factor_capitalizacion.append(fc)
            i=i+1
           
        factor_capitalizacion.append(get_wf(self.start_date[1],self.start_date[0],self.fixings[0]))
        self.P=producto(factor_capitalizacion)
        self.diff_dias = self.start_date[len(self.start_date)-1].day_diff(self.start_date[0])
        self.tasa_eq = (self.P-1) * 360.0 / self.diff_dias
        return self.tasa_eq
    
    
    def get_accrued_interest(self, start_date: Qcf.QCDate, fixings) -> float:
        self.notional = 10000000
        self.interes_flot_ois = (self.notional*self.tasa_eq*self.diff_dias) / 360
        return self.interes_flot_ois
    
    def amount(self, fixings) -> float:
        self.flujo_total=self.interes_flot_ois+self.notional
        return self.flujo_total 

    

In [13]:
r1=OIS()
start_date=r1.start_date=data["fechas"]
fixings=r1.fixings=data["fixing"]
#Llamado a la primera función de la clase
print("La tasa equivalente es:",r1.get_accrued_rate(start_date,fixings))
#Llamado a lasegunda función de la clase
print("El interes es:",r1.get_accrued_interest(start_date,fixings))
#Llamado a la tercera funciónd e la clase
print("El monto es:",r1.amount(fixings))

La tasa equivalente es: 0.00696242230123633
El interes es: 76006.44345516327
El monto es: 10076006.443455163


Construcción de un objeto `Qcf.InterestRateIndex`.

In [14]:
codigo = 'SOFR'
tasa_on = Qcf.QCInterestRate(.0, Qcf.QCAct360(), Qcf.QCLinearWf())
fixing_lag = Qcf.Tenor('0d')
tenor = Qcf.Tenor('1d')
fixing_calendar = get_cal(BusCal.NY)     
settlement_calendar = fixing_calendar  
sofr = Qcf.InterestRateIndex(
    codigo,
    tasa_on,
    fixing_lag,
    tenor,
    fixing_calendar,
    settlement_calendar,
    Qcf.QCUSD()
)

Construcción de una instancia de `OisCashflow`.

In [15]:
ois = OIS(
    Qcf.QCDate(1, 10, 2019),
    Qcf.QCDate(28, 10, 2020),
    Qcf.QCDate(1, 10, 2019),
    10000000,
    Qcf.QCUSD(),
    1000000,
    True,
    Qcf.QCInterestRate(0.0, Qcf.QCAct360(), Qcf.QCLinearWf()),
    sofr,
    0,
    1
)

TypeError: OIS() takes no arguments

In [16]:
try:
    ois.test1(0)
except Exception as e:
    print('Hacer algo')

Hacer algo


## Funciones

In [17]:
# Data
df_curva = pd.read_excel("../data/20201012_built_sofr_zero.xlsx")
df_curva

,plazo,tasa,df
0,1,0.000811,0.999998
1,7,0.000841,0.999984
2,14,0.000780,0.999970
3,21,0.000774,0.999955
4,33,0.000781,0.999929
5,61,0.000781,0.999870
6,92,0.000811,0.999796
7,125,0.000781,0.999733
8,152,0.000760,0.999683
9,182,0.000750,0.999626


In [18]:
#Formato de la tabla
frmt = {
    'nominal': '{:,.2f}',
    'amortizacion': '{:,.2f}',
    'interes': '{:,.2f}',
    'flujo': '{:,.2f}',
    'interes_cap': '{:,.2f}',
    'interes_total': '{:,.2f}',
    'valor_tasa': '{:.6%}',
    'tasa': '{:.6%}'
}

In [19]:
def get_ois():
    get_ois_sofr = partial(aux.get_ois_using_template, aux.type_ois_template, aux.TypeOis.SOFR)
    return get_ois_sofr

In [20]:
def set_expected_rate(
        val_date: Qcf.QCDate,
        ois_cashflow: OIS,
        zcc: Qcf.ZeroCouponCurve,
        fixings: Qcf.time_series) -> None:
    
    fwd = Qcf.ForwardRates()
    get=get_ois()   
    op = get(rp=Qcf.RecPay.RECEIVE,notional=10000000,start_date=Qcf.QCDate(1, 6, 2019),tenor=Qcf.Tenor('2Y'),fixed_rate_value=.01,spread=0,gearing=1) 
    i=0
    while i< len(fixings):
        operaciones = fwd.set_rates_icp_clp_leg(val_date, fixings[i], op[1], zcc)
        i=i+1
    return aux.show_leg(op[1], 'IcpClpCashflow', '').style.format(frmt)

In [21]:
zcc = aux.get_curve_from_dataframe(Qcf.QCAct365(),Qcf.QCCompoundWf(), df_curva)
val_date=Qcf.QCDate(2, 10, 2020)
fixing=data["fixing"]
Tasas_esperadas=set_expected_rate(val_date, OIS, zcc, fixing)
Tasas_esperadas

,fecha_inicial,fecha_final,fecha_pago,nominal,amortizacion,amort_es_flujo,flujo,moneda,icp_inicial,icp_final,valor_tasa,interes,spread,gearing,tipo_tasa
0,2019-06-01,2020-06-01,2020-06-01,"-10,000,000.00",0.00,True,0.00,CLP,1.000000,1.000000,0.000000%,-0.00,0.000000,1.000000,LinAct360
1,2020-06-01,2021-06-01,2021-06-01,"-10,000,000.00","-10,000,000.00",True,"-188,090.37",CLP,1.000000,0.018809,-96.770000%,"9,811,402.78",0.000000,1.000000,LinAct360


In [22]:
def present_value(val_date,ois_cashflow,zcc) -> float:
    Tasa =zcc.get_rate_at(plazo)
    DF=zcc.get_discount_factor_at(plazo)
    OIS = partial(aux.get_ois_using_template, aux.type_ois_template, aux.TypeOis.SOFR)
    OP = OIS(rp=Qcf.RecPay.RECEIVE,notional=10000000,start_date=Qcf.QCDate(1, 9, 2020),tenor=Qcf.Tenor('2Y'),fixed_rate_value=.01,spread=0.0,gearing=1.0)
    valor_presente = Qcf.PresentValue()
    fecha= val_date
    Forward = Qcf.ForwardRates()
    Forward.set_rates_icp_clp_leg(fecha, 1.0, OP[1], zcc)
    vp_flot = valor_presente.pv(fecha, OP[1], zcc)
    vp_fija = valor_presente.pv(fecha, OP[0], zcc)
    valor_total=vp_fija+vp_flot
    return valor_total

In [23]:
#Para la función 5 
zcc = aux.get_curve_from_dataframe(Qcf.QCAct365(),Qcf.QCCompoundWf(), df_curva)
plazo = 730
Tasa_730 =zcc.get_rate_at(plazo)
DF=zcc.get_discount_factor_at(plazo)
val_date=Qcf.QCDate(28, 10, 2020)
present_value(val_date,OIS,zcc)

191725.23899785243

## Tests

In [24]:
import sys
sys.path.insert(1, '../modules')
import auxiliary as aux

In [25]:
amount_tol = 10000
rate_tol = .001

In [26]:
exitos = 0

In [27]:
def suma_exito(resultado, check, tipo):
    global exitos
    if resultado is None:
        return
    else:
        print('Suma medio punto por obtener resultado.')
        exitos += .5
        if tipo == 'monto':
            if abs(resultado - check) < amount_tol:
                print('Suma 1 punto por obtener resultado dentro de la tolerancia.')
                exitos += 1
        else:
            if abs(resultado - check) < rate_tol:
                print('Suma 1 punto por obtener resultado dentro de la tolerancia.')
                exitos += 1

Construcción de una instancia de `OisCashflow`.

In [28]:
ois = OIS()

In [29]:
#ois = OisCashflow(
#    Qcf.QCDate(1, 10, 2019),
#    Qcf.QCDate(1, 10, 2020),
#    Qcf.QCDate(1, 10, 2020),
#    10000000,
#    Qcf.QCUSD(),
#    1000000,
#    True,
#    Qcf.QCInterestRate(0.0, Qcf.QCAct360(), Qcf.QCLinearWf()),
#    sofr,
#    0,
#    1,
#    settlement_calendar
#)

AD: Al instanciar de esta forma el objeto OIS, éste no contiene data alguna. Como el método `__init__` está mal declarado, la clase ni siquiera tiene las variables internas donde guardar la data.
Además de no seguir las instrucciones, esto obliga a modificar la clase antes de poder usar sus métodos.

En la siguiente variable `exitos` se registra cuantos tests se superan con éxito.

### Objeto `fixings`

In [30]:
df_fixings = pd.read_excel('../data/SOFR-10012019-10292020.xls', sheet_name='nice format')
df_fixings.columns = ['fecha', 'nombre', 'valor']
df_fixings['valor'] /= 100

In [31]:
df_fixings.head().style.format({'valor': '{:.4%}'})

,fecha,nombre,valor
0,2020-10-28,SOFR,0.0800%
1,2020-10-27,SOFR,0.0900%
2,2020-10-26,SOFR,0.0900%
3,2020-10-23,SOFR,0.0800%
4,2020-10-22,SOFR,0.0700%


In [32]:
fixings = Qcf.time_series()
for row in df_fixings.itertuples():
    fixings[Qcf.build_qcdate_from_string(row.fecha)] = row.valor

In [33]:
fixings[Qcf.QCDate(13, 1, 2020)]

0.0154

### `OisCashflow.accrued_rate`

In [34]:
accrued_rate = ois.get_accrued_rate(Qcf.QCDate(15, 6, 2020), fixings)
print(f'accrued rate: {accrued_rate:.8%}')

AttributeError: 'OIS' object has no attribute 'start_date'

AD: Se aplica test entregado por el grupo:

- Se debe modificar la clase para poder usarla.
- Los tipos de variables usados en los métodos no corresponden a las instrucciones y son inconsistentes con las declaraciones de los mismos.

In [35]:
ois.start_date=data["fechas"]
ois.fixings=data["fixing"]
accrued_rate = ois.get_accrued_rate(data["fechas"],fixings)
print("La tasa equivalente es:",accrued_rate)

La tasa equivalente es: 0.00696242230123633


Se verificará el cálculo usando `df_fixings`.

In [36]:
df_fixings.head().style.format({'valor': '{:.4%}'})

,fecha,nombre,valor
0,2020-10-28,SOFR,0.0800%
1,2020-10-27,SOFR,0.0900%
2,2020-10-26,SOFR,0.0900%
3,2020-10-23,SOFR,0.0800%
4,2020-10-22,SOFR,0.0700%


Se agrega la columna `next_date`, servirá para calcular el factor de capitalización de cada tasa.

In [37]:
def next_date(fecha:str, calendario: Qcf.BusinessCalendar) -> str:
    qfecha = Qcf.build_qcdate_from_string(fecha)
    next_fecha = calendario.shift(qfecha, 1)
    return next_fecha.description(False)

In [38]:
df_fixings['next_fecha'] = df_fixings.apply(
    lambda row: next_date(row['fecha'], settlement_calendar),
    axis=1
)

In [39]:
df_fixings.head().style.format({'valor': '{:.4%}'})

,fecha,nombre,valor,next_fecha
0,2020-10-28,SOFR,0.0800%,2020-10-29
1,2020-10-27,SOFR,0.0900%,2020-10-28
2,2020-10-26,SOFR,0.0900%,2020-10-27
3,2020-10-23,SOFR,0.0800%,2020-10-26
4,2020-10-22,SOFR,0.0700%,2020-10-23


Se calcula ahora el factor de capitalización.

In [40]:
def factor_cap(fecha: str, next_fecha: str, valor: float) -> float:
    qfecha = Qcf.build_qcdate_from_string(fecha)
    qnext_fecha = Qcf.build_qcdate_from_string(next_fecha)
    int_rate = Qcf.QCInterestRate(valor, Qcf.QCAct360(), Qcf.QCLinearWf())
    return int_rate.wf(qfecha, qnext_fecha)

In [41]:
df_fixings['factor_capitalizacion'] = df_fixings.apply(
    lambda row: factor_cap(row['fecha'], row['next_fecha'], row['valor']),
    axis=1
)

In [42]:
df_fixings.head().style.format({'valor': '{:.4%}'})

,fecha,nombre,valor,next_fecha,factor_capitalizacion
0,2020-10-28,SOFR,0.0800%,2020-10-29,1.000002
1,2020-10-27,SOFR,0.0900%,2020-10-28,1.000002
2,2020-10-26,SOFR,0.0900%,2020-10-27,1.000002
3,2020-10-23,SOFR,0.0800%,2020-10-26,1.000007
4,2020-10-22,SOFR,0.0700%,2020-10-23,1.000002


Veamos donde están en `df_fixings` los valores para `start_date` de `ois` y la fecha 15-6-2020 y calculamos el producto de los factores de capitalización desde esa fecha hasta la última fecha de `df_flujos` (que es el primer registro). Luego, el cociente de los dos factores nos dará el factor entre ambas fechas. Finalmente, con ese factor, se calculará `accrued_rate`.

AD: se modifica este cálculo ya que la clase OIS no se comporta según lo instruido.

In [43]:
# start_date = ois.start_date.description(False)
start_date = Qcf.QCDate(1, 10, 2019).description(False)
df_fixings[df_fixings.fecha == start_date]

,fecha,nombre,valor,next_fecha,factor_capitalizacion
269,2019-10-01,SOFR,0.0188,2019-10-02,1.000052


In [44]:
factor_largo = df_fixings.iloc[:270]['factor_capitalizacion'].prod()
print(factor_largo)
qstart_date = Qcf.build_qcdate_from_string('2019-10-01')

1.0075715527341267


In [45]:
df_fixings[df_fixings.fecha == '2020-06-15']

,fecha,nombre,valor,next_fecha,factor_capitalizacion
94,2020-06-15,SOFR,0.0009,2020-06-16,1.000002


In [46]:
factor_corto = df_fixings.iloc[:95]['factor_capitalizacion'].prod()
qaccrued_date = Qcf.build_qcdate_from_string('2020-06-15')

In [47]:
dias = qstart_date.day_diff(qaccrued_date)
check_accrued_rate = (factor_largo / factor_corto - 1.0) * 360.0 / dias
print(f'La diferencia es: {accrued_rate - check_accrued_rate:.8%}')

La diferencia es: -0.31367091%


In [48]:
try:
    suma_exito(accrued_rate, check_accrued_rate, 'tasa')
except Exception as e:
    print('No hay resultado.')

Suma medio punto por obtener resultado.


### `OisCashflow.accrued_interest`

In [49]:
accrued_interest = ois.get_accrued_interest(Qcf.QCDate(15, 6, 2020), fixings)
print(f'accrued interest: {accrued_interest:,.2f}')

accrued interest: 76,006.44


In [50]:
check_accrued_interest = ois.notional * accrued_rate * dias / 360.0
print(f'La diferencia es: {accrued_interest - check_accrued_interest:,.6f}')

La diferencia es: 26,109.083630


In [51]:
try:
    suma_exito(accrued_rate, check_accrued_rate, 'tasa')
except Exception as e:
    print('No hay resultado.')

Suma medio punto por obtener resultado.


### `OisCashflow.amount`

In [52]:
amount = ois.amount(fixings)
print(f'amount: {amount:,.2f}')

amount: 10,076,006.44


In [53]:
end_date = ois.end_date.description(False)
print(f'end date: {end_date}')
df_fixings[df_fixings.fecha == end_date]

AttributeError: 'OIS' object has no attribute 'end_date'

AD: se modifica el test porque la clase no corresponde ni a lo instruido ni a lo declarado.

In [54]:
ois.end_date = Qcf.QCDate(1, 10, 2020)
end_date = ois.end_date.description(False)
print(f'end date: {end_date}')
df_fixings[df_fixings.fecha == end_date]

end date: 2020-10-01


,fecha,nombre,valor,next_fecha,factor_capitalizacion
18,2020-10-01,SOFR,0.0008,2020-10-02,1.000002


In [55]:
ois.notional = 10000000 # AD: se modifica la clase
ois.amortization = 1000000 # AD: se modifica la clase
factor_corto_2 = df_fixings.iloc[:19]['factor_capitalizacion'].prod()
dias_2 = qstart_date.day_diff(ois.end_date)
accrued_rate_amount = (factor_largo / factor_corto_2- 1.0) * 360.0 / dias_2
check_amount = ois.notional * accrued_rate_amount * dias_2 / 360.0 + ois.amortization
print(f'La diferencia es: {amount - check_amount:.6f}')

La diferencia es: 9000979.398022


In [56]:
try:
    suma_exito(accrued_rate, check_accrued_rate, 'tasa')
except Exception as e:
    print('No hay resultado.')

Suma medio punto por obtener resultado.


### `set_expected_rate`

Comenzamos cargando los valores de la curva cero cupón que se utilizará para hacer pruebas de `set_expected_rate` y `present_value`.

In [57]:
df_curva = pd.read_excel('../data/20201012_built_sofr_zero.xlsx')

In [58]:
df_curva.head().style.format({'tasa': '{:.4%}', 'df': '{:.6%}'})

,plazo,tasa,df
0,1,0.0811%,99.999778%
1,7,0.0841%,99.998388%
2,14,0.0780%,99.997010%
3,21,0.0774%,99.995549%
4,33,0.0781%,99.992942%


Con la data se construye un objeto de tipo `Qcf.ZeroCouponCurve`.

In [59]:
zcc = aux.get_curve_from_dataframe(Qcf.QCAct365(),Qcf.QCCompoundWf(), df_curva)

Comienzan los tests.

In [60]:
print(f'Fecha inicial flujo: {ois.start_date.description(False)}')
print(f'Fecha final flujo: {ois.end_date.description(False)}')

AttributeError: 'Series' object has no attribute 'description'

In [61]:
# Check primer caso
# AD: se modifica según test entregado por el grupo
val_date = Qcf.QCDate(1, 6, 2019)
set_expected_rate(val_date, ois, zcc, data["fixing"])
resultado = ois.interest_rate.get_value()
print(f'Tasa esperada es: {resultado:.8%}')
p1 = val_date.day_diff(ois.start_date)
p2 = val_date.day_diff(ois.end_date)
df1 = zcc.get_discount_factor_at(p1)
df2 = zcc.get_discount_factor_at(p2)
check = (df1 / df2 - 1) * 360 / (p2 - p1)
print(f'Check tasa esperada es: {check:.8%}')

suma_exito(resultado, check, 'tasa')

AttributeError: 'OIS' object has no attribute 'interest_rate'

In [62]:
# Check segundo caso borde
val_date = Qcf.QCDate(1, 10, 2019)
set_expected_rate(val_date, ois, fixings, zcc)
resultado = ois.interest_rate.get_value()
print(f'Tasa esperada es: {resultado:.8%}')
p1 = val_date.day_diff(ois.start_date)
p2 = val_date.day_diff(ois.end_date)
df1 = zcc.get_discount_factor_at(p1)
df2 = zcc.get_discount_factor_at(p2)
check = (df1 / df2 - 1) * 360 / (p2 - p1)
print(f'Check tasa esperada es: {check:.8%}')

suma_exito(resultado, check, 'tasa')

TypeError: object of type 'ZeroCouponCurve' has no len()

In [63]:
# Check segundo caso
val_date = Qcf.QCDate(1, 11, 2019)
set_expected_rate(val_date, ois, fixings, zcc)
resultado = ois.interest_rate.get_value()
print(f'Tasa esperada es: {resultado:.8%}')
p1 = ois.start_date.day_diff(val_date)
p2 = val_date.day_diff(ois.end_date)
wf1 = 1 + ois.get_accrued_interest(val_date, fixings) / ois.notional
wf2 = 1 / zcc.get_discount_factor_at(p2)
check = (wf1 * wf2 - 1) * 360 / (p1 + p2)
print(f'Check tasa esperada es: {check:.8%}')

suma_exito(resultado, check, 'tasa')

TypeError: object of type 'ZeroCouponCurve' has no len()

In [64]:
# Check tercer caso borde
val_date = Qcf.QCDate(1, 10, 2020)
set_expected_rate(val_date, ois, fixings, zcc)
resultado = ois.interest_rate.get_value()
print(f'Tasa esperada es: {resultado:.8%}')
check = ois.get_accrued_rate(val_date, fixings)
print(f'Check tasa esperada es: {check:.8%}')

suma_exito(resultado, check, 'tasa')

TypeError: object of type 'ZeroCouponCurve' has no len()

In [65]:
# Check tercer caso
val_date = Qcf.QCDate(15, 10, 2020)
set_expected_rate(val_date, ois, fixings, zcc)
resultado = ois.interest_rate.get_value()
print(f'Tasa esperada es: {resultado:.8%}')
check = ois.get_accrued_rate(ois.end_date, fixings)
print(f'Check tasa esperada es: {check:.8%}')

suma_exito(resultado, check, 'tasa')

TypeError: object of type 'ZeroCouponCurve' has no len()

### `present_value`

In [66]:
# Check primer caso
val_date = Qcf.QCDate(1, 6, 2019)
set_expected_rate(val_date, ois, fixings, zcc)
pv = present_value(val_date, ois, zcc)
print(f'Valor presente es: {pv:,.4f}')
amount = ois.amount()
print(f'Amount: {amount:,.4f}')
amount = ois.amount()
plazo = val_date.day_diff(ois.settlement_date)
df = zcc.get_discount_factor_at(plazo)
print(f'Df: {df:.6%}')
vp_check = df * amount
print(f'VP check es: {vp_check:,.4f}')

suma_exito(pv, vp_check, 'monto')

TypeError: object of type 'ZeroCouponCurve' has no len()

In [67]:
# Check segundo caso borde
val_date = Qcf.QCDate(1, 10, 2019)
set_expected_rate(val_date, ois, fixings, zcc)
pv = present_value(val_date, ois, zcc)
print(f'Valor presente es: {pv:,.4f}')
amount = ois.amount()
print(f'Amount: {amount:,.4f}')
amount = ois.amount()
plazo = val_date.day_diff(ois.settlement_date)
df = zcc.get_discount_factor_at(plazo)
print(f'Df: {df:.6%}')
vp_check = df * amount
print(f'VP check es: {vp_check:,.4f}')

suma_exito(pv, vp_check, 'monto')

TypeError: object of type 'ZeroCouponCurve' has no len()

In [68]:
# Check segundo caso
val_date = Qcf.QCDate(1, 11, 2019)
set_expected_rate(val_date, ois, fixings, zcc)
pv = present_value(val_date, ois, zcc)
print(f'Valor presente es: {pv:,.4f}')
amount = ois.amount()
print(f'Amount: {amount:,.4f}')
amount = ois.amount()
plazo = val_date.day_diff(ois.settlement_date)
df = zcc.get_discount_factor_at(plazo)
print(f'Df: {df:.6%}')
vp_check = df * amount
print(f'VP check es: {vp_check:,.4f}')

suma_exito(pv, vp_check, 'monto')

TypeError: object of type 'ZeroCouponCurve' has no len()

In [69]:
# Check tercer caso borde
val_date = Qcf.QCDate(1, 10, 2020)
set_expected_rate(val_date, ois, fixings, zcc)
pv = present_value(val_date, ois, zcc)
print(f'Valor presente es: {pv:,.4f}')
amount = ois.amount()
print(f'Amount: {amount:,.4f}')
amount = ois.amount()
plazo = val_date.day_diff(ois.settlement_date)
if plazo <= 0:
    df = 0.0
else:
    df = zcc.get_discount_factor_at(plazo)
print(f'Df: {df:.6%}')
vp_check = df * amount
print(f'VP check es: {vp_check:,.4f}')

suma_exito(pv, vp_check, 'monto')

TypeError: object of type 'ZeroCouponCurve' has no len()

In [70]:
# Check tercer caso
val_date = Qcf.QCDate(15, 10, 2020)
set_expected_rate(val_date, ois, fixings, zcc)
pv = present_value(val_date, ois, zcc)
print(f'Valor presente es: {pv:,.4f}')
amount = ois.amount()
print(f'Amount: {amount:,.4f}')
amount = ois.amount()
plazo = val_date.day_diff(ois.settlement_date)
if plazo <= 0:
    df = 0.0
else:
    df = zcc.get_discount_factor_at(plazo)
print(f'Df: {df:.6%}')
vp_check = df * amount
print(f'VP check es: {vp_check:,.4f}')

suma_exito(pv, vp_check, 'monto')

TypeError: object of type 'ZeroCouponCurve' has no len()

In [71]:
print(f'Éxitos totales: {exitos}')

Éxitos totales: 1.5
